# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: lora 
* Model: gpt-2
* Evaluation approach: Trainer.evaluate()
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [2]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",
    num_labels=6,
    id2label={0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"},  
    label2id={"sadness": 0, "joy": 1, "love": 2, "anger": 3, "fear": 4, "surprise": 5},)
# Freeze all the base model parameters.
for param in model.parameters():
    param.requires_grad = False
model.classifier
print(model)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [3]:
from datasets import load_dataset

splits = ["train", "validation", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset("dair-ai/emotion", split=splits))}
print(ds)

ds["train"] = ds["train"].shuffle(seed=21).select(range(6000))
ds["validation"] = ds["validation"].shuffle(seed=22).select(range(2000))
ds["test"] = ds["train"].shuffle(seed=23).select(range(1000))
    
ds

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


def preprocess_function(examples):
    """Preprocess the dataset by returning tokenized examples."""
    encoded = tokenizer(examples['text'], padding='max_length', truncation=True)
    return encoded


tokenized_ds = {}
for split in ds.keys():
    tokenized_ds[split] = ds[split].map(preprocess_function, batched=True)
    
print(tokenized_ds["train"][0]["input_ids"])

{'train': Dataset({
    features: ['text', 'label'],
    num_rows: 16000
}), 'validation': Dataset({
    features: ['text', 'label'],
    num_rows: 2000
}), 'test': Dataset({
    features: ['text', 'label'],
    num_rows: 2000
})}
[101, 1045, 2123, 1056, 2514, 2061, 6091, 2725, 2047, 2477, 4902, 1045, 2031, 2062, 1997, 2019, 2023, 2003, 2054, 1045, 2031, 2000, 2079, 1998, 1045, 2097, 2079, 2009, 2828, 1997, 7729, 2738, 2084, 2019, 1045, 2428, 3246, 1045, 2123, 2102, 11224, 2039, 2828, 1997, 7729, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [4]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis",
        learning_rate=1e-2,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=5,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Evaluate the base model
trainer.evaluate()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 1.804656982421875,
 'eval_accuracy': 0.156,
 'eval_runtime': 18.3356,
 'eval_samples_per_second': 54.539,
 'eval_steps_per_second': 1.745}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [5]:
from peft import LoraConfig, TaskType, get_peft_model

peft_config = LoraConfig(task_type='SEQ_CLS',target_modules={'classifier'}, r=8, lora_alpha=32, lora_dropout=0.1)

peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 1,202,796 || all params: 67,565,676 || trainable%: 1.780187916716766


In [11]:
training_args = TrainingArguments(
    output_dir="./data/peft_analysis",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="no",
#    load_best_model_at_end=True,
#    save_safetensors=False,
)

peft_trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

In [12]:
peft_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.177004,0.549500
2,No log,1.126444,0.579000


TrainOutput(global_step=376, training_loss=1.1519106601146942, metrics={'train_runtime': 285.9045, 'train_samples_per_second': 41.972, 'train_steps_per_second': 1.315, 'total_flos': 1612120375296000.0, 'train_loss': 1.1519106601146942, 'epoch': 2.0})

In [14]:
peft_model.save_pretrained("./models/peft_model")

TypeError: Object of type set is not JSON serializable

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.